# EDA de **Telefonía móvil: Accesos Totales**

Cuaderno preparado para ejecutarse en **Visual Studio/VS Code** con Python. Incluye pasos guiados y visualizaciones básicas.
**Fuente de datos:** CSV con columnas `Registro_ID, Año, Trimestre, Total de accesos pospago, Total de accesos prepago, Total de accesos operativos, TiempoTrim_ID`.

## 1. Importaciones y configuración
- Se importan `pandas`, `numpy`, `matplotlib` y `seaborn`.
- Se configura el estilo de gráficos y se asegura codificación UTF-8.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:** `C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Accesos Totales_con_ID.csv`

Si tu usuario o carpeta cambian, actualiza la cadena `ruta`. Se preserva la codificación **UTF‑8 con BOM** para abrir acentos correctamente en Excel.

In [ ]:
ruta = r"C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Accesos Totales_con_ID.csv"

# Cargar el archivo desde la ruta de Windows indicada
df = pd.read_csv(ruta, encoding='utf-8-sig')

# Copia de seguridad del crudo
df_raw = df.copy()

# Vista rápida
display(df.head())
print('
Filas, Columnas:', df.shape)
print('
Columnas:', list(df.columns))

## 3. Perfil inicial del data set
- Tipos de datos y memoria (`df.info()`).
- Valores faltantes y duplicados.
- Rango básico de las métricas.

In [ ]:
# Tipos de datos
df.info()

# Valores faltantes y duplicados
print('
Valores faltantes por columna:
', df.isna().sum())
print('
Duplicados (filas completas):', df.duplicated().sum())

# Rango básico
numeric_cols = [
    'Registro_ID','Año','Trimestre',
    'Total de accesos pospago','Total de accesos prepago','Total de accesos operativos',
    'TiempoTrim_ID'
]
for col in numeric_cols:
    if col in df.columns:
        print(f'
[{col}] min={df[col].min()} max={df[col].max()} mean={df[col].mean():.2f}')

## 4. Validaciones de consistencia
- Conversión de columnas numéricas.
- Chequeo de negativos.
- Verificación de que **pospago + prepago = operativos**.

In [ ]:
# Conversión a numérico
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Chequeo de valores negativos
negativos = (df[numeric_cols] < 0).sum()
print('
Conteo de negativos por columna:
', negativos)

# Consistencia: pospago + prepago vs operativos
if set(['Total de accesos pospago','Total de accesos prepago','Total de accesos operativos']).issubset(df.columns):
    suma = df['Total de accesos pospago'] + df['Total de accesos prepago']
    consistentes = (suma == df['Total de accesos operativos'])
    print(f'
Registros consistentes: {consistentes.mean():.2%} ({consistentes.sum()} de {len(consistentes)})')
    # Guardar diferencia para inspección
    df['dif_operativos'] = df['Total de accesos operativos'] - suma

## 5. Limpieza y ordenamiento
- Eliminación de duplicados.
- Orden cronológico por `Año` y `Trimestre`.
- Remoción de columnas auxiliares si no se requieren.

In [ ]:
df = df.drop_duplicates()
df = df.sort_values(['Año','Trimestre']).reset_index(drop=True)

# Remover columnas auxiliares si existen
df = df.drop(columns=['dif_operativos'], errors='ignore')

display(df.head())

## 6. Variables derivadas
Se calculan porcentajes de participación de **pospago** y **prepago** sobre el total operativo.

In [ ]:
if set(['Total de accesos pospago','Total de accesos prepago','Total de accesos operativos']).issubset(df.columns):
    df['porc_pospago'] = df['Total de accesos pospago'] / df['Total de accesos operativos'] * 100
    df['porc_prepago'] = df['Total de accesos prepago'] / df['Total de accesos operativos'] * 100

display(df[['Año','Trimestre','porc_pospago','porc_prepago']].head())

## 7. Visualizaciones
- **Serie temporal** de `Total de accesos operativos`.
- **Evolución porcentual** de pospago vs prepago.

In [ ]:
import matplotlib.ticker as mtick

# Construir etiqueta de periodo YYYY-Qn para el eje X
if set(['Año','Trimestre']).issubset(df.columns):
    periodo = pd.PeriodIndex(year=df['Año'], quarter=df['Trimestre']).astype(str)
    df['Periodo'] = periodo

# 7.1 Tendencia de accesos operativos
a = 'Total de accesos operativos'
if 'Periodo' in df.columns and a in df.columns:
    plt.figure()
    sns.lineplot(data=df, x='Periodo', y=a, marker='o', color='#2E86C1')
    plt.xticks(rotation=45, ha='right')
    plt.title('Accesos operativos por trimestre')
    plt.xlabel('Periodo')
    plt.ylabel('Accesos operativos')
    plt.tight_layout()
    plt.show()

# 7.2 Porcentaje pospago vs prepago (últimos 12 trimestres)
if {'porc_pospago','porc_prepago','Periodo'}.issubset(df.columns):
    tail = df.tail(12)
    plt.figure()
    sns.lineplot(data=tail, x='Periodo', y='porc_pospago', marker='o', label='Pospago', color='#1ABC9C')
    sns.lineplot(data=tail, x='Periodo', y='porc_prepago', marker='o', label='Prepago', color='#E67E22')
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
    plt.xticks(rotation=45, ha='right')
    plt.title('Participación (%) pospago vs prepago (últimos 12 trimestres)')
    plt.xlabel('Periodo')
    plt.ylabel('% del total de accesos operativos')
    plt.legend()
    plt.tight_layout()
    plt.show()

## 8. Guardado del archivo limpio
Se guarda el CSV **en la misma ruta** con la **leyenda `_limpio`** agregada al nombre, conservando la codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)